In [ ]:
%pip install torch transformers pandas scikit-learn accelerate

# %pip install --upgrade transformers torch accelerate

# %pip show transformers torch accelerate

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# load dataset
df = pd.read_csv('/content/reskill_dataset_v1.csv')

# split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

train_df.head()

,resume_text,skills
2821,mobile +61 469 778 024 email rahulsharma9431@g...,"amazon web services (aws), azure sentinel, car..."
1963,anupama b. pradeep anupamapradeep111@gmail.com...,"salesforce.com, salesforce, sfd, asp.net, boom..."
7,nikit juneja c l o u d e n g i n e e r s e n i...,"informatica, powercenter, teradata, oracle, un..."
693,ayman salaheldeen adam mohammed it system admi...,"microsoft azure, azure virtual machines, azure..."
1429,job description role devops engineer duration ...,"git, msbuild, maven, nuget, tsqlt, selenium, a..."


In [ ]:
# extract skills from the dataset
all_skills = df['skills'].apply(lambda x: [skill.strip() for skill in x.split(',')])

# flatten the list of lists and find unique skills
unique_skills = set(skill for sublist in all_skills for skill in sublist)

unique_skills = list(unique_skills)

print(f"Unique skills: {unique_skills[:10]}")
print(f"Number of unique skills: {len(unique_skills)}")

Unique skills: ['apex trigger', 'itil v2', 'multiple threads', 'failover cluster', 'agile development', 'kronos accruals', 'fluke', 'windows)', 'uat testing', 'angular material']
Number of unique skills: 18857


In [ ]:
from transformers import BertTokenizer

# load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_data(df):
    return tokenizer(
        df['resume_text'].tolist(),
        padding=True,
        truncation=True,
        return_tensors='pt'
    )

# tokenize the training and validation data
train_encodings = tokenize_data(train_df)
val_encodings = tokenize_data(val_df)

train_encodings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

{'input_ids': tensor([[  101,  4684,  1009,  ...,  7621,  1998,   102],
        [  101,  2019,  6279,  ...,  5604,  1998,   102],
        [  101, 23205,  4183,  ...,  6521,  2000,   102],
        ...,
        [  101,  7187,  3748,  ...,  2449,  3208,   102],
        [  101,  8882, 19300,  ..., 27390,  2229,   102],
        [  101,  6670,  7556,  ...,  5896,  2241,   102]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [ ]:
import torch
import numpy as np
from sklearn.preprocessing import LabelEncoder
from transformers import BertForTokenClassification, Trainer, TrainingArguments
import torch.nn as nn

# Assuming you have a list of all unique skills
all_skills = list(set([skill for sublist in train_labels for skill in sublist]))

# Create a mapping from skills to indices
skill_to_index = {skill: idx for idx, skill in enumerate(all_skills)}

# Convert skills to token-level labels
def encode_labels_token_level(skill_list, all_skills, max_length=512):
    encoded = np.full(max_length, -100)  # Use -100 for padding
    for skill in skill_list:
        if skill in skill_to_index:
            encoded[skill_to_index[skill]] = 1  # Mark presence of the skill
    return encoded.tolist()

# Update the encoding function for train and validation sets
train_labels_encoded = train_df['skills'].apply(
    lambda x: encode_labels_token_level(
        [skill.strip() for skill in x.split(',')],
        all_skills
    )
).tolist()

val_labels_encoded = val_df['skills'].apply(
    lambda x: encode_labels_token_level(
        [skill.strip() for skill in x.split(',')],
        all_skills
    )
).tolist()

In [ ]:
import torch
from sklearn.preprocessing import MultiLabelBinarizer

# initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# fit and transform the labels
train_labels = mlb.fit_transform(train_df['skills'].apply(lambda x: [skill.strip() for skill in x.split(',')]))
val_labels = mlb.transform(val_df['skills'].apply(lambda x: [skill.strip() for skill in x.split(',')]))

class ResumeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}

        # Create a tensor for labels
        label_tensor = torch.tensor(self.labels[idx])

        # Pad the labels to match the input length
        if len(label_tensor) < len(item['input_ids']):
            padding_length = len(item['input_ids']) - len(label_tensor)
            label_tensor = torch.cat((label_tensor, torch.full((padding_length,), -100)))  # -100 is ignored by the loss function

        item['labels'] = label_tensor
        return item

    def __len__(self):
        return len(self.labels)

# create dataset objects
# train_dataset = ResumeDataset(train_encodings, train_labels)
# val_dataset = ResumeDataset(val_encodings, val_labels)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['.ini', '.net 5 and 6', '.net core 2.0', '.net core 2.1', '.net core mvc', '.net development', '.net framework 4.5 to 4.7.2', '.xlsx', '10i', '11.2', '12', '124', '125', '134', '135', '16.2', '175', '1800 1300 inbound platform', '18c', '204', '205', '215', '225', '26xx', '275', '2850', '2g', '2k', '305', '315', '3400', '367', '37xx', '380', '3810m', '3d printing', '3ds', '3fa', '3gpp rel-11', '3gpps', '4.1', '4.2', '4.4.5', '5.6', '520 hb2', '520 hb3 blade servers', '5400zl)', '5g technology strategy', '5k', '6-sigma', '6.x', '6000', '6500 series)', '65xx', '6800', '7.2', '70-431', '7200 series)', '7210', '7240xm', '77', '7k', '8051 microprocessor assembly', '8200', '8600', '8600 series switches', '8i', '9.5', '9.x', '903', 'a/b split testing', 'ac/dc drives', 'ac/vc troubleshooting', 'accelq', 'access 95', 'access control system', 'access manager', 'account reconciliation recon

In [ ]:
train_dataset[10]

{'input_ids': tensor([  101,  1051,  1038,  1046,  1041,  1039,  1056,  1045,  1058,  1041,
          4813,  3076,  2007,  2051,  2968,  1998, 12317,  2764, 22676,  1998,
          6951,  2083,  3454,  2007,  2204,  3834,  4281,  2003,  6224,  2000,
          2707,  1037,  2476,  1012,  9657,  2551,  2007,  8146,  7578,  8578,
          1998,  3716,  3085,  1998, 10326,  4619,  2007,  9373,  4824,  1998,
          2070,  6742,  3325,  1012,  1039,  1051,  1050,  1056,  1037,  1039,
          1056,  5754,  1012,  2703, 24096,  1030,  5840,  2575,  2620, 24434,
         17788,  2692,  2475, 20917,  4014,  1012,  4012, 16770,  1013,  1013,
          7479,  1012,  4957,  2063, 11586,  1012,  4012,  1013,  1999,  1013,
          5754,  1011,  2984,  1011,  2703,  1011,  6365,  2692, 16086,  2683,
         16932,  2620,  1013,  2171,  5754,  2984,  2703,  5907,  2931, 10662,
          2796,  1041,  1040,  1057,  1039,  1037,  1056,  1045,  1051,  1050,
          5065,  2015,  2297,  1011,  2

In [ ]:
# Load the model with the correct number of labels
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(all_skills))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # Use BCEWithLogitsLoss for multi-label classification
        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits.view(-1, logits.size(-1)), labels.float().view(-1, labels.size(-1)))  # Flatten the tensors

        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='model/test_v1',
    evaluation_strategy='epoch',
    logging_dir='model/test_v1/logs',
    logging_steps=10,
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

ValueError: Target size (torch.Size([16, 15925])) must be the same as input size (torch.Size([8192, 2]))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.plot(trainer.train_losses, label='Train Loss')
plt.plot(trainer.eval_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()